In [14]:
import json
import pandas as pd
import numpy as np
from emoji import core
import re
import ast
from matplotlib import pyplot as plt

In [15]:
playlists = pd.read_json('../0_data/playlists.json', typ = 'frame', encoding='utf-8')
song_meta = pd.read_json('../0_data/song_meta.json', typ = 'frame', encoding='utf-8') 
genre_gn_all = pd.read_json('../0_data/genre_gn_all.json', typ = 'frame', encoding='utf-8', orient='index')

### 0. genre

In [16]:
genre_gn_all.head(10)

,0
GN0100,발라드
GN0101,세부장르전체
GN0102,'80
GN0103,'90
GN0104,'00
GN0105,'10-
GN0200,댄스
GN0201,세부장르전체
GN0202,'80
GN0203,'90


In [17]:
# 컬럼명 지정
genre_gn_all = genre_gn_all.reset_index().rename(columns={'index':'gnr_code', 0:'gnr_name'})
genre_gn_all.head()

# 대분류 장르코드들만 뽑기
gnr_code = genre_gn_all[genre_gn_all['gnr_code'].str[-2:]=='00']

# 소분류 장르코드만 뽑기
dtl_gnr_code = genre_gn_all[genre_gn_all['gnr_code'].str[-2:] != '00']
dtl_gnr_code.rename(columns = {'gnr_code' : 'dtl_gnr_code', 'gnr_name' : 'dtl_gnr_name'}, inplace = True)

# 대분류 장르코드와 소분류 장르코드 각각의 앞자리 네자리 공통코드 추출
gnr_code = gnr_code.assign(join_code = gnr_code['gnr_code'].str[0:4])
dtl_gnr_code = dtl_gnr_code.assign(join_code = dtl_gnr_code['dtl_gnr_code'].str[0:4])

# join_code(네자리 공통코드) 기준으로 Merge
gnr_code_tree = pd.merge(gnr_code, dtl_gnr_code, how = 'left', on = 'join_code')

gnr_code_tree[['gnr_code', 'gnr_name', 'dtl_gnr_code', 'dtl_gnr_name']]
gnr_code_tree

gnr_big = pd.DataFrame(gnr_code.iloc[:, :2])

C:\Users\cg063\AppData\Local\Temp\ipykernel_13660\3749102257.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtl_gnr_code.rename(columns = {'gnr_code' : 'dtl_gnr_code', 'gnr_name' : 'dtl_gnr_name'}, inplace = True)


In [ ]:
# song_meta 저장
gnr_big.to_json('../0_data/genre_gn_all_filter.json', orient='records')

### 1. song_meta

> 0. album name이 비어 있는 노래 삭제

In [ ]:
song_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707989 entries, 0 to 707988
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   song_gn_dtl_gnr_basket  707989 non-null  object
 1   issue_date              707989 non-null  int64 
 2   album_name              707985 non-null  object
 3   album_id                707989 non-null  int64 
 4   artist_id_basket        707989 non-null  object
 5   song_name               707989 non-null  object
 6   song_gn_gnr_basket      707989 non-null  object
 7   artist_name_basket      707989 non-null  object
 8   id                      707989 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 48.6+ MB


In [18]:
song_meta[song_meta['album_name'].isnull()] # album_name 비어 있는 경우 확인

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
143209,"[GN0501, GN0601, GN0503, GN0606, GN0509]",20150522,None,2318271,[726168],사랑바보,"[GN0500, GN0600]",[민은서밴드],143209
291631,"[GN0105, GN0101]",20101108,None,1065178,[28772],빗속에서,[GN0100],[존박],291631
431110,"[GN0509, GN0601, GN0503, GN0606, GN0501]",20200208,None,10385717,[2203064],화면,"[GN0500, GN0600]",[Oowl Hannal (우울 한날)],431110
552088,"[GN0105, GN0601]",20101108,None,1065178,[28263],본능적으로 (Feat. Swings),"[GN0600, GN0100]",[강승윤],552088


In [19]:
# 바꾸려는 앨범명 리스트
new_album_names = ["사랑바보", "슈퍼스타K 2 Top11 Part.5", "null", "슈퍼스타K 2 Top11 Part.3"]

# 바꾸기
song_meta.loc[song_meta['album_name'].isnull(), 'album_name'] = new_album_names

In [20]:
# 확인
song_meta.iloc[[143209, 291631, 431110, 552088], :]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
143209,"[GN0501, GN0601, GN0503, GN0606, GN0509]",20150522,사랑바보,2318271,[726168],사랑바보,"[GN0500, GN0600]",[민은서밴드],143209
291631,"[GN0105, GN0101]",20101108,슈퍼스타K 2 Top11 Part.5,1065178,[28772],빗속에서,[GN0100],[존박],291631
431110,"[GN0509, GN0601, GN0503, GN0606, GN0501]",20200208,null,10385717,[2203064],화면,"[GN0500, GN0600]",[Oowl Hannal (우울 한날)],431110
552088,"[GN0105, GN0601]",20101108,슈퍼스타K 2 Top11 Part.3,1065178,[28263],본능적으로 (Feat. Swings),"[GN0600, GN0100]",[강승윤],552088


> 1. 날짜 오류 수정

In [ ]:
# 날짜 오류있는 행 확인
song_meta[song_meta['issue_date'] > 20220000] # index : 141185, 448286

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
141185,[GN1801],20220113,고요한 밤바다 구경하기 [여수 바다],10403230,[2737142],잔잔한 바람이 불어와 (There is a gentle breeze),[GN1800],[무드홀릭 (Moodholic)],141185
448286,[GN1801],20220113,고요한 밤바다 구경하기 [여수 바다],10403230,[2737142],여수 바다 (Yeosu sea),[GN1800],[무드홀릭 (Moodholic)],448286


In [21]:
indices_to_update = [141185, 448286]
new_value = 20200113 # 알맞는 발매날짜
# 특정 행의 'issue_date' 값을 변경
for idx in indices_to_update:
    song_meta.at[idx, 'issue_date'] = new_value

In [ ]:
# 알맞게 변경되었는지 확인
song_meta.iloc[[141185, 448286], :]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
141185,[GN1801],20200113,고요한 밤바다 구경하기 [여수 바다],10403230,[2737142],잔잔한 바람이 불어와 (There is a gentle breeze),[GN1800],[무드홀릭 (Moodholic)],141185
448286,[GN1801],20200113,고요한 밤바다 구경하기 [여수 바다],10403230,[2737142],여수 바다 (Yeosu sea),[GN1800],[무드홀릭 (Moodholic)],448286


> 2.song_gn_dtl_gnr_basket가 비어있는 경우와 대분류 장르 GN9000 삭제

In [22]:
song_meta['song_gn_dtl_gnr_basket_cnt'] = song_meta['song_gn_dtl_gnr_basket'].apply(len)
song_meta[song_meta['song_gn_dtl_gnr_basket_cnt']==0]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,song_gn_dtl_gnr_basket_cnt
21,[],20200331,WHY,10410508,[417985],WHY,[],[4minute],21,0
262,[],20150318,노년의 건강을 지켜주는 음악 처방전 (부제 : 휴식 때 들으면 좋은 음악친구),2309524,[726909],Bio Sound `Breathing & Big Bell`,[GN9000],[차병원],262,0
399,[],20200106,뽀로로 겨울 동요,10372098,[1703695],초코 초코 핫초코,[],[아이코닉스 (ICONIX)],399,0
454,[],20191108,MAGDALENE,10349014,[788767],thousand eyes,[GN1100],[FKA Twigs],454,0
803,[],20040901,Tropical Lullaby,153435,[848353],Kalua Lullaby,[],[The Moonlighters],803,0
...,...,...,...,...,...,...,...,...,...,...
706502,[],20110831,엄마와 아가와의 행복한 대화 태교음악 뉴에이지,2009854,[561734],Love Love Love,[GN9000],[해피타임뮤직],706502,0
706716,[],20200109,Have It All,10373946,[2762722],Have It All,[GN9000],[Renegade],706716,0
707201,[],20200115,모두의 MR반주 123,10376105,[770787],Into the Unknown (겨울왕국 2 OST) (Melody MR),[GN9000],[모두의MR],707201,0
707759,[],20200225,낭만닥터 김사부 2 OST,10393897,"[27619, 726783]",Emergency,[],"[전창엽, 마마고릴라]",707759,0


In [23]:
# 소분류 장르가 없는 song id 담아두기
song_gnr_empty = list(song_meta[song_meta['song_gn_dtl_gnr_basket_cnt'] == 0]['id'].values) # 4037개
# 장르정보가 GN9000인 song id 담아두기
song_gnr_GN9000 = list(song_meta[song_meta['song_gn_gnr_basket'].apply(lambda x: 'GN9000' in x)]['id'].values) # 1834개

# 삭제될 song id list
to_del_song_id = set(song_gnr_empty + song_gnr_GN9000)
len(to_del_song_id) # 4040개

4040

In [24]:
song_meta = song_meta[song_meta['song_gn_dtl_gnr_basket_cnt'] != 0] # 소분류 장르가 없는 song 4037개 삭제
song_meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 703952 entries, 0 to 707988
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   song_gn_dtl_gnr_basket      703952 non-null  object
 1   issue_date                  703952 non-null  int64 
 2   album_name                  703952 non-null  object
 3   album_id                    703952 non-null  int64 
 4   artist_id_basket            703952 non-null  object
 5   song_name                   703952 non-null  object
 6   song_gn_gnr_basket          703952 non-null  object
 7   artist_name_basket          703952 non-null  object
 8   id                          703952 non-null  int64 
 9   song_gn_dtl_gnr_basket_cnt  703952 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 59.1+ MB


In [25]:
song_meta = song_meta[~song_meta['song_gn_gnr_basket'].apply(lambda x: 'GN9000' in x)] # GN9000가 있는 행 3개삭제 (GN9000이 대분류 장르인 경우, 대부분 소분류 정보가 없어서 여기서는 3개만 삭제됨)
song_meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 703949 entries, 0 to 707988
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   song_gn_dtl_gnr_basket      703949 non-null  object
 1   issue_date                  703949 non-null  int64 
 2   album_name                  703949 non-null  object
 3   album_id                    703949 non-null  int64 
 4   artist_id_basket            703949 non-null  object
 5   song_name                   703949 non-null  object
 6   song_gn_gnr_basket          703949 non-null  object
 7   artist_name_basket          703949 non-null  object
 8   id                          703949 non-null  int64 
 9   song_gn_dtl_gnr_basket_cnt  703949 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 59.1+ MB


In [26]:
# 장르 정보가 없는 행이 없는지 확인
song_meta[song_meta['song_gn_dtl_gnr_basket_cnt']==0]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,song_gn_dtl_gnr_basket_cnt


In [27]:
# GN9000가 삭제되었는지 확인
song_meta[song_meta['song_gn_gnr_basket'].apply(lambda x: 'GN9000' in x)]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,song_gn_dtl_gnr_basket_cnt


> 3. 대분류 장르명과 song_meta merge

In [28]:
song_meta_ex = song_meta.explode('song_gn_gnr_basket') # 대분류 장르 한줄에 하나씩 넣기
song_meta_ex.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,song_gn_dtl_gnr_basket_cnt
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,GN0900,[Various Artists],0,1
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",GN1600,[Murray Perahia],1,2
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),GN0900,[Peter Gabriel],2,1
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,GN1100,[Matoma],3,2
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,GN1800,[Jude Law],4,2


In [29]:
# 대분류 장르와 song_meta merge
song_meta_merge = pd.merge(song_meta_ex, gnr_big, how='left', left_on='song_gn_gnr_basket', right_on='gnr_code')
song_meta_merge.drop('gnr_code', axis=1, inplace=True)
song_meta_merge.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,song_gn_dtl_gnr_basket_cnt,gnr_name
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,GN0900,[Various Artists],0,1,POP
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",GN1600,[Murray Perahia],1,2,클래식
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),GN0900,[Peter Gabriel],2,1,POP
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,GN1100,[Matoma],3,2,일렉트로니카
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,GN1800,[Jude Law],4,2,뉴에이지


In [30]:
song_meta_gnr_code = song_meta_merge.groupby('id')['song_gn_gnr_basket'].apply(list).reset_index() # 하나의 song에 대해 하나의 대분류 장르코드 list로 만들기
song_meta_gnr_name = song_meta_merge.groupby('id')['gnr_name'].apply(list).reset_index() # 하나의 song에 대해 하나의 대분류 장르명 list로 만들기

In [31]:
song_meta_f = pd.merge(song_meta_merge, song_meta_gnr_code, how='left', on='id') # 장르코드 list와 merge
song_meta_f = pd.merge(song_meta_f, song_meta_gnr_name, how='left', on='id') # 장르명 list와 merge

In [32]:
# 필요한 컬럼만 남기기
song_meta_f = song_meta_f[['song_gn_dtl_gnr_basket', 'issue_date', 'album_name', 'album_id', 'artist_id_basket', 'song_name', 'song_gn_gnr_basket_y', 'gnr_name_y', 'artist_name_basket', 'id']]
# 컬럼명 수정
song_meta_f.rename(columns={'song_gn_gnr_basket_y':'song_gn_gnr_basket', 'gnr_name_y':'gnr_name'}, inplace=True)

song_meta_f.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,gnr_name,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[POP],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[클래식],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[POP],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[일렉트로니카],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[뉴에이지],[Jude Law],4


In [33]:
song_meta_cleaned = song_meta_f.drop_duplicates(subset='id', keep='first') # id가 중복인 행에 대해 하나의 id행만 남기기

In [34]:
song_meta_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 703949 entries, 0 to 799772
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   song_gn_dtl_gnr_basket  703949 non-null  object
 1   issue_date              703949 non-null  int64 
 2   album_name              703949 non-null  object
 3   album_id                703949 non-null  int64 
 4   artist_id_basket        703949 non-null  object
 5   song_name               703949 non-null  object
 6   song_gn_gnr_basket      703949 non-null  object
 7   gnr_name                703949 non-null  object
 8   artist_name_basket      703949 non-null  object
 9   id                      703949 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 59.1+ MB


In [ ]:
# song_meta 저장
song_meta_cleaned.to_json('../0_data/orgi_song_meta.json', orient='records')

> 4. song_meta DB용으로 만들기

In [35]:
song_meta_db = song_meta_cleaned[['id', 'song_name', 'artist_name_basket', 'gnr_name']]
song_meta_db.rename(columns={'id':'song_id', 'artist_name_basket':'artist_name_lst', 'gnr_name':'song_gnr_lst'}, inplace=True)

C:\Users\cg063\AppData\Local\Temp\ipykernel_13660\593262896.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  song_meta_db.rename(columns={'id':'song_id', 'artist_name_basket':'artist_name_lst', 'gnr_name':'song_gnr_lst'}, inplace=True)


In [36]:
song_meta_db.head()

,song_id,song_name,artist_name_lst,song_gnr_lst
0,0,Feelings,[Various Artists],[POP]
1,1,"Bach : Partita No. 4 In D Major, BWV 828 - II....",[Murray Perahia],[클래식]
2,2,Solsbury Hill (Remastered 2002),[Peter Gabriel],[POP]
3,3,Feeling Right (Everything Is Nice) (Feat. Popc...,[Matoma],[일렉트로니카]
4,4,그남자 그여자,[Jude Law],[뉴에이지]


In [37]:
# db_song_meta 저장
song_meta_db.to_json('../0_data/db_song_meta.json', orient='records')

### 2. playlists

> 0. 장르정보, GN9000대분류 장르였던 노래들 삭제

In [38]:
len(to_del_song_id)

4040

In [39]:
# song_meta에서 삭제한 to_del_song_id들을 playlists에서도 삭제
playlists['songs'] = playlists['songs'].apply(lambda x: [song for song in x if song not in to_del_song_id])

In [40]:
# songs가 비어 있는 행이 있는지 확인 -> 10개(아기 클래식, 뮤지컬 동화, 자장가 등)
playlists[playlists['songs'].apply(lambda x: len(x) == 0)] 

# 비어 있는 행은 삭제
playlists = playlists[playlists['songs'].apply(lambda x: len(x) != 0)] 

In [41]:
playlists.info() # 10행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115061 entries, 0 to 115070
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   tags          115061 non-null  object
 1   id            115061 non-null  int64 
 2   plylst_title  115061 non-null  object
 3   songs         115061 non-null  object
 4   like_cnt      115061 non-null  int64 
 5   updt_date     115061 non-null  object
dtypes: int64(2), object(4)
memory usage: 6.1+ MB


> 1. 타이틀, 태그 불용어 처리

- 타이틀 특수문자 제거

In [42]:
# 이모지 제거 함수
def replace_emoji(inputString):
    return core.replace_emoji(inputString, replace='')

# 특수문자 제거 함수
def clean_text(inputString):
  text_rmv = re.sub('[-=+,#/\?:^●○.@*\"※~ㆍ!<>$♥❤♡☆★♪♬♩¶♨①๑｡＊②ఇ¥◈ ͡✔ ͡° ͜ʖ▶◀◎ღ✿ﾉ ;\\\_■『│』ᕷⓗ‘|\(\)\[\]`\'…》\”\“\’·]',' ', inputString)
  return text_rmv

In [43]:
# # 이모지 제거 함수 적용
playlists['plylst_title'] = playlists['plylst_title'].apply(replace_emoji)

# 특수문자 제거 함수 적용
playlists['plylst_title'] = playlists['plylst_title'].apply(clean_text)

In [44]:
# 양 옆 공백, %, & 제거

playlists['plylst_title'] = playlists['plylst_title'].str.strip()
playlists['plylst_title'] = playlists['plylst_title'].str.strip('%')
playlists['plylst_title'] = playlists['plylst_title'].str.strip('&')

In [45]:
playlists[playlists['id'].isin([22525, 143039])]

,tags,id,plylst_title,songs,like_cnt,updt_date
56134,[흥폭발],22525,내적댄스 터져베이베 아이돌 노래노래,"[108003, 594076, 297411, 97189, 35713, 445680,...",41,2018-06-15 19:24:26.000
59162,"[가을, 설렘, 사랑]",143039,가을이다 o 곧 겨울이다 T T,"[13142, 5049, 503816, 70166, 578529, 336829, 2...",1,2018-07-31 10:55:04.000


- 태그 특수문자 제거

In [46]:
# 특수문자 '_'가 있는 행 찾기
# '_'가 있는 행 찾기
rows_with_underscore = playlists['tags'][playlists['tags'].apply(lambda x: any('_' in tag for tag in x))]
rows_with_underscore

148                            [비오는날, 비_오는_날, 휴식, 비_오는날, 비]
285                                [상쾌해지는, 호불호없는, 모두가_흥얼흥얼]
364        [추천, 일렉트로니카, 5월_4주차, 주간, 최신, 전지음악, 인기, 주간일렉트로니카]
409                      [들었으면, 인디음악, 한번쯤, 이미지는_직접찍은사진, 인디]
522       [힙합, label_crew, 랩, 국내힙합, 트렌드, 쇼미더머니, 스웨그, aom...
                                ...                        
114625                                   [칠_Chill, 설레는_달콤함]
114647                     [우울, 위로, 위로가_필요할_때, 슬픔, 이별, 힘든날]
114688    [어쿠스틱, 혼자, 발라드, 우울, 분위기, 멜로디, 띵곡들, 조용한, 자기_전, 인디]
114703    [패션위크, 뉴욕, 도시, 갈라쇼음악, 트렌디, 팝, 편집샵, i_know_it, ...
115068               [담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]
Name: tags, Length: 2252, dtype: object

In [47]:
# 특수 문자, 빈 태그를 제거하는 함수 정의
def remove_special_characters(tag_list):
    cleaned_tags = []
    for tag in tag_list:
        cleaned_tag = re.sub(r'[^\uAC00-\uD7A30-9a-zA-Z\s]', '', tag).replace(" ", "")
        if cleaned_tag:
            cleaned_tags.append(cleaned_tag)
    return cleaned_tags

# 함수 적용
playlists['tags'] = playlists['tags'].apply(remove_special_characters)

In [48]:
playlists[playlists['id'].isin([32533, 151357, 11343, 95032])]

,tags,id,plylst_title,songs,like_cnt,updt_date
148,"[비오는날, 비오는날, 휴식, 비오는날, 비]",32533,비오는날 감성 자극 듣기 좋은 어쿠스틱 피아노 발라드,"[133091, 527764, 468136, 260894, 22845, 148876...",14,2020-02-16 00:56:45.000
285,"[상쾌해지는, 호불호없는, 모두가흥얼흥얼]",151357,기분좋아지는 노래들,"[270061, 158766, 573179, 272062, 37027, 245028...",0,2019-11-24 05:49:55.000
56921,"[힙합, 흑인음악, 힙합엘이, 힙합추천, 틱톡, 외힙, HIPHOPLE]",95032,어떻게 떴냐고 답은 바로 TikTok,"[253222, 270262, 301533, 136007, 238347, 13384...",7,2020-01-08 11:19:11.000
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의이야기, 사랑, 친구]",11343,1 눈물이 앞을 가리는 나의 이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000


In [49]:
# 플레이리스트 타이틀이 1글자인 조건 만들기
playlists_title_1str = playlists[playlists['plylst_title'].str.len() == 1]
playlists_title_1str = list(playlists_title_1str['id'].values) # 24 -> 롹, 잠, 쿨, 힙 등이며 대부분 좋아요 0 또는 1
# playlists[playlists['id'] == 98628]

# 삭제
playlists = playlists[playlists['plylst_title'].str.len() != 1]

In [50]:
# 빈 타이틀 삭제
playlists = playlists[playlists['plylst_title'] != '']

# 숫자로만 구성된 타이틀 삭제
numeric_title_rows = playlists[playlists['plylst_title'].str.isnumeric()]
indices_to_remove = numeric_title_rows.index
playlists = playlists.drop(indices_to_remove)

playlists.info()


# 115061  - 114697  = 364행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114697 entries, 0 to 115070
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   tags          114697 non-null  object
 1   id            114697 non-null  int64 
 2   plylst_title  114697 non-null  object
 3   songs         114697 non-null  object
 4   like_cnt      114697 non-null  int64 
 5   updt_date     114697 non-null  object
dtypes: int64(2), object(4)
memory usage: 6.1+ MB


> 2. 타이틀 중복 제거

In [51]:
# 1. 플레이리스트 아이디(id)와 플레이리스트 명(plylst_title) 추출
plylst_title = playlists[['id', 'plylst_title']]

# 2. 플레이리스트 이름 별 플레이리스트 ID count 테이블 생성 : plylst_id_cnt
plylst_id_cnt = plylst_title.groupby('plylst_title').id.nunique().reset_index(name = 'mapping_plylst_cnt')

# 3. 플레이리스트 명 중복 구분 : 서로 다른 플레이리스트 아이디로 겹치는 플레이리스트 이름이 없으면 '중복 없음', 그 외 '중복 있음'
plylst_id_cnt = plylst_id_cnt.assign(
    mapping_plylst_cnt_category = pd.cut(plylst_id_cnt['mapping_plylst_cnt'], [0, 1, np.inf], labels = ['중복 없음', '중복 있음'])
)
plylst_id_cnt
# [0, 1, np.inf] : 0이상 1미만 값 = 중복없음, 1이상의 값 = 중복있음

# 4. 중복 구분 별 플레이리스트 수 count 테이블 생성 : plylst_id_cnt_division
plylst_id_cnt_division = pd.DataFrame(plylst_id_cnt.groupby('mapping_plylst_cnt_category').count())
plylst_id_cnt_division

,plylst_title,mapping_plylst_cnt
mapping_plylst_cnt_category,,
중복 없음,106515,106515
중복 있음,2722,2722


In [52]:
# 플레이리스트 타이틀을 기준으로 merge해 중복된 플레이리스트만 확인
playlists_dup = pd.merge(playlists, plylst_id_cnt, on='plylst_title')
playlists_dup= playlists_dup[playlists_dup['mapping_plylst_cnt_category'] == '중복 있음']
playlists_dup['song_cnt'] = playlists_dup['songs'].apply(len)
playlists_dup['tag_cnt'] = playlists_dup['tags'].apply(len)
playlists_dup.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt,tag_cnt
16,"[감성, 질리지않는, 나만알고싶은, Pop]",1516,나만 알고싶은 노래들,"[331055, 99287, 310974, 376435, 146989, 430106...",1,2018-01-02 12:10:59.000,4,중복 있음,31,4
17,"[기분전환, 까페]",86179,나만 알고싶은 노래들,"[480142, 578916, 467225, 701072, 197046, 63557...",6,2016-10-06 02:08:40.000,4,중복 있음,46,2
18,"[숨은명곡, 좋은노래]",54673,나만 알고싶은 노래들,"[113618, 422482, 380069, 75425, 79141, 657137,...",7,2016-07-15 10:20:17.000,4,중복 있음,23,2
19,"[감성힙합, 나만알고싶은, 뿜뿜, 그루브, 노래, 감성]",135616,나만 알고싶은 노래들,"[470548, 603591, 382764, 288501, 187291, 37802...",30,2018-07-13 08:18:54.000,4,중복 있음,45,6
44,"[기분전환, 가을]",105328,트렌디한 Cafe Pop Music,"[419702, 403579, 472776, 502078, 662300, 44363...",55,2014-09-18 11:29:55.000,2,중복 있음,16,2


In [53]:
# 같은 타이틀을 가진 플레이리스트의 노래 개수 비교
# playlists_dup에서 타이틀별로 groupby 후 song_cnt 값들을 song_cnt_nunique(새 컬럼)에 리스트 형태로 넣은 후 중복을 없애기 위해 set 진행
# 1의 의미는 노래 갯수가 중복되어 있다는 뜻이기 때문에 1의 갯수가 적은 지표를 기준으로 중복 플레이리스트 제거를 진행해야함
from collections import Counter

playlists_dup_song_cnt = playlists_dup.groupby('plylst_title')['song_cnt'].apply(list).reset_index()
playlists_dup_song_cnt['song_cnt_nunique'] = playlists_dup_song_cnt['song_cnt'].apply(lambda x: len(set(x)))
Counter(playlists_dup_song_cnt['song_cnt_nunique'])

Counter({2: 1544,
         1: 507,
         3: 344,
         4: 131,
         5: 58,
         6: 45,
         7: 30,
         9: 6,
         14: 5,
         20: 2,
         8: 20,
         15: 5,
         11: 7,
         10: 6,
         16: 3,
         13: 3,
         25: 1,
         12: 2,
         19: 1,
         22: 1,
         17: 1})

In [54]:
# 같은 타이틀을 가진 플레이리스트의 태그 개수 비교

playlists_dup_tag_cnt = playlists_dup.groupby('plylst_title')['tag_cnt'].apply(list).reset_index()
playlists_dup_tag_cnt['tag_cnt_nunique'] = playlists_dup_tag_cnt['tag_cnt'].apply(lambda x: len(set(x)))
Counter(playlists_dup_tag_cnt['tag_cnt_nunique'])

Counter({1: 1225, 3: 183, 2: 1222, 4: 59, 5: 20, 6: 9, 7: 4})

In [55]:
# 같은 타이틀을 가진 플레이리스트의 좋아요 비교

playlists_dup_like_cnt = playlists_dup.groupby('plylst_title')['like_cnt'].apply(list).reset_index()
playlists_dup_like_cnt['like_cnt_nunique'] = playlists_dup_like_cnt['like_cnt'].apply(lambda x: len(set(x)))
Counter(playlists_dup_like_cnt['like_cnt_nunique'])

Counter({1: 431,
         2: 1741,
         3: 311,
         4: 102,
         5: 54,
         6: 32,
         7: 14,
         11: 4,
         9: 7,
         13: 2,
         10: 4,
         12: 5,
         8: 10,
         15: 1,
         14: 2,
         18: 1,
         24: 1})

- 플레이리스트 타이틀이 중복인 플레이리스트만 따로 모으고
- 그 중복 플리모음 중 노래갯수가 가장 많은 하나만 남겨두고
- 원래 train의 중복 없음만 따로 모아 중복모음과 concat

In [56]:
# train에 중복 갯수, 중복 여부 컬럼 추가, 플레이리스트 내 곡 갯수 추가
playlists_add_col = pd.merge(playlists, plylst_id_cnt, on='plylst_title')
playlists_add_col['song_cnt'] = playlists_add_col['songs'].apply(lambda x : len(x))

# 매핑갯수, 타이틀, 플리 내 노래 갯수, like_cnt 내림차순 정렬
playlists_filter = playlists_add_col.sort_values(['mapping_plylst_cnt', 'plylst_title', 'song_cnt', 'like_cnt'], ascending=False)
playlists_filter

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt
8531,"[집중, 어쿠스틱, 혼자, 발라드, 우울, 드라이브, 산책, 조용조용, 띵곡들, 인디]",152785,혼자만의 시간이 필요할때,"[208812, 629738, 232874, 13281, 193232, 389468...",1,2020-03-26 06:14:54.000,43,중복 있음,81
8518,"[어쿠스틱, 혼자, 발라드, 우울, 분위기, 드라이브, 띵곡만, 산책, 운전, 인디]",127180,혼자만의 시간이 필요할때,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-03-24 06:39:20.000,43,중복 있음,78
8535,"[띵곡, 혼자만, 어쿠스틱, 발라드, 우울, 분위기, 드라이브, 조용한, 산책, 인디]",76448,혼자만의 시간이 필요할때,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-04-13 05:56:55.000,43,중복 있음,78
8540,"[어쿠스틱, 혼자, 발라드, 우울, 분위기, 드라이브, 조용한, 산책, 띵곡들, 인디]",141114,혼자만의 시간이 필요할때,"[484964, 626369, 570836, 366786, 413837, 20913...",1,2020-04-22 06:38:57.000,43,중복 있음,78
8547,"[어쿠스틱, 혼자, 발라드, 분위기, 드라이브, 조용한, 산책, 운전, 띵곡들, 인디]",18201,혼자만의 시간이 필요할때,"[484964, 626369, 570836, 366786, 413837, 20913...",1,2020-04-06 05:20:46.000,43,중복 있음,78
...,...,...,...,...,...,...,...,...,...
31841,[일렉],56144,00 Party 일렉트로댄스,"[256418, 183424, 431998, 654442, 354592, 24132...",7,2018-02-06 16:02:42.000,1,중복 없음,20
17803,[일렉],58768,00 Electronica EDM,"[673003, 251403, 617690, 441958, 436128, 62788...",5,2020-01-05 00:24:22.000,1,중복 없음,133
108575,[일렉],110536,0 일렉듣고 광질준비 O K ElecVVIP,"[349306, 93890, 651902, 46845, 362347, 330768,...",173,2012-11-29 14:34:56.000,1,중복 없음,59
52777,"[비오는날, 추억, 회상]",106660,비오는날 듣기 좋은 연주곡 모음,"[79329, 51922, 173875, 216115, 422584, 509605,...",6,2015-10-06 11:45:24.000,1,중복 없음,25


In [57]:
playlists_dup = playlists_filter[playlists_filter['mapping_plylst_cnt_category'] == '중복 있음'] # 중복 플리 모음
playlists_dup['plylst_title'].nunique() # 중복된 플레이리스트의 유니크 타이틀 갯수 : 2722

2722

In [58]:
playlists_dup['tag_cnt'] = playlists_dup['tags'].apply(len)
playlists_dup.head()

C:\Users\cg063\AppData\Local\Temp\ipykernel_13660\2448186990.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playlists_dup['tag_cnt'] = playlists_dup['tags'].apply(len)


,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt,tag_cnt
8531,"[집중, 어쿠스틱, 혼자, 발라드, 우울, 드라이브, 산책, 조용조용, 띵곡들, 인디]",152785,혼자만의 시간이 필요할때,"[208812, 629738, 232874, 13281, 193232, 389468...",1,2020-03-26 06:14:54.000,43,중복 있음,81,10
8518,"[어쿠스틱, 혼자, 발라드, 우울, 분위기, 드라이브, 띵곡만, 산책, 운전, 인디]",127180,혼자만의 시간이 필요할때,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-03-24 06:39:20.000,43,중복 있음,78,10
8535,"[띵곡, 혼자만, 어쿠스틱, 발라드, 우울, 분위기, 드라이브, 조용한, 산책, 인디]",76448,혼자만의 시간이 필요할때,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-04-13 05:56:55.000,43,중복 있음,78,10
8540,"[어쿠스틱, 혼자, 발라드, 우울, 분위기, 드라이브, 조용한, 산책, 띵곡들, 인디]",141114,혼자만의 시간이 필요할때,"[484964, 626369, 570836, 366786, 413837, 20913...",1,2020-04-22 06:38:57.000,43,중복 있음,78,10
8547,"[어쿠스틱, 혼자, 발라드, 분위기, 드라이브, 조용한, 산책, 운전, 띵곡들, 인디]",18201,혼자만의 시간이 필요할때,"[484964, 626369, 570836, 366786, 413837, 20913...",1,2020-04-06 05:20:46.000,43,중복 있음,78,10


In [59]:
# playlists_dup 중 각 플리 제목 별 노래갯수가 가장 많은 하나만 남긴다
playlists_dup_filter = pd.DataFrame(playlists_dup[playlists_dup.groupby('plylst_title')['song_cnt'].transform(max) == playlists_dup['song_cnt']]) # 3572 
playlists_dup_filter = pd.DataFrame(playlists_dup_filter[playlists_dup_filter.groupby('plylst_title')['like_cnt'].transform(max) == playlists_dup_filter['like_cnt']]) # 2928 
playlists_dup_filter = pd.DataFrame(playlists_dup_filter[playlists_dup_filter.groupby('plylst_title')['tag_cnt'].transform(max) == playlists_dup_filter['tag_cnt']]) # 2909 
playlists_dup_filter = pd.DataFrame(playlists_dup_filter[playlists_dup_filter.groupby('plylst_title')['updt_date'].transform(max) == playlists_dup_filter['updt_date']]) # 2748 
playlists_dup_filter

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt,tag_cnt
8531,"[집중, 어쿠스틱, 혼자, 발라드, 우울, 드라이브, 산책, 조용조용, 띵곡들, 인디]",152785,혼자만의 시간이 필요할때,"[208812, 629738, 232874, 13281, 193232, 389468...",1,2020-03-26 06:14:54.000,43,중복 있음,81,10
3119,[비오는날듣기좋은노래],75012,비오는날,"[65672, 522825, 145252, 146780, 500731, 473608...",22,2017-08-19 15:16:36.000,34,중복 있음,200,1
5536,"[조용히, 어쿠스틱, 여유, 발라드, 분위기, 힐링, 자기전이나, 띵곡, 인디, 편안한]",52312,자기전 하루한곡 발라드,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-04-02 06:54:25.000,31,중복 있음,82,10
8428,[금요일],48129,금요일,"[614756, 147427, 340332, 675466, 414036, 61619...",0,2017-03-10 17:31:34.000,30,중복 있음,20,1
7287,[발라드],5960,추억,"[208821, 481749, 565402, 362197, 407236, 46604...",10,2010-10-21 09:45:12.000,27,중복 있음,85,1
...,...,...,...,...,...,...,...,...,...,...
155,"[기분좋은, 100번]",148495,100번 들어도 기분좋은 가요,"[208186, 155952, 669617, 438439, 397412, 60398...",4,2016-10-31 13:59:41.000,2,중복 있음,16,2
76630,[Pop],62278,100% 로맨틱 조지 마이클 223,"[366485, 435804, 442680, 538949, 263360, 29495...",0,2019-03-27 15:27:44.000,2,중복 있음,34,1
34439,[팝],50564,1 HITS OF DECADE SPECIAL 2,"[60025, 326802, 82991, 533287, 404318, 82686, ...",5,2009-07-11 21:37:42.000,2,중복 있음,37,1
75844,[팝],109558,1 HITS OF DECADE SPECIAL,"[289618, 78629, 349460, 218356, 189583, 613209...",5,2009-07-05 18:23:47.000,2,중복 있음,39,1


In [60]:
playlists_dup_filter[playlists_dup_filter['plylst_title'] == 'qqqa'] # 필터링 되지 않은 중복 타이틀은 플레이리스트 자체가 같음

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt,tag_cnt
50436,[여름],7524,qqqa,"[522825, 455668, 549178, 407828, 343974]",1,2015-11-04 17:51:14.000,3,중복 있음,5,1
50437,[여름],43031,qqqa,"[522825, 455668, 549178, 407828, 343974]",1,2015-11-04 17:51:14.000,3,중복 있음,5,1


In [61]:
# 타이틀이 같은 플레이리스트 중 하나만 남긴다
playlists_dup_filter = pd.DataFrame(playlists_dup_filter.groupby('plylst_title', group_keys=False).apply(lambda x: x.loc[x['song_cnt'].idxmax()]))
playlists_dup_filter = playlists_dup_filter.reset_index(drop=True) # 2722

In [62]:
# playlists_dup_filter 중 중복 여부 확인

# 1. 플레이리스트 아이디(id)와 플레이리스트 명(plylst_title) 추출
plylst_title_2 = playlists_dup_filter[['id', 'plylst_title']]

# 2. 플레이리스트 이름 별 플레이리스트 ID count 테이블 생성 : plylst_id_cnt
plylst_id_cnt_2 = plylst_title_2.groupby('plylst_title').id.nunique().reset_index(name = 'mapping_plylst_cnt')

# 3. 플레이리스트 명 중복 구분 : 서로 다른 플레이리스트 아이디로 겹치는 플레이리스트 이름이 없으면 '중복 없음', 그 외 '중복 있음'
plylst_id_cnt_2 = plylst_id_cnt_2.assign(
    mapping_plylst_cnt_category = pd.cut(plylst_id_cnt_2['mapping_plylst_cnt'], [0, 1, np.inf], labels = ['중복 없음', '중복 있음'])
)
plylst_id_cnt_2
# [0, 1, np.inf] : 0이상 1미만 값 = 중복없음, 1이상의 값 = 중복있음

# 4. 중복 구분 별 플레이리스트 수 count 테이블 생성 : plylst_id_cnt_division
plylst_id_cnt_division_2 = pd.DataFrame(plylst_id_cnt_2.groupby('mapping_plylst_cnt_category').count())
plylst_id_cnt_division_2

,plylst_title,mapping_plylst_cnt
mapping_plylst_cnt_category,,
중복 없음,2722,2722
중복 있음,0,0


In [63]:
# 중복 없음 파일만 만들기
playlists_no_dup = pd.merge(playlists, plylst_id_cnt, on='plylst_title')
playlists_no_dup = playlists_no_dup[playlists_no_dup['mapping_plylst_cnt_category'] == '중복 없음']
playlists_no_dup # 106515행

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,1,중복 없음
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,1,중복 없음
2,"[까페, 잔잔한]",76951,편하게 잔잔하게 들을 수 있는 곡,"[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,1,중복 없음
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,1,중복 없음
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,1,중복 없음
...,...,...,...,...,...,...,...,...
114692,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E SM 2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000,1,중복 없음
114693,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000,1,중복 없음
114694,"[담시, 가족, 눈물, 그리움, 주인공, 나의이야기, 사랑, 친구]",11343,1 눈물이 앞을 가리는 나의 이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000,1,중복 없음
114695,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000,1,중복 없음


In [64]:
playlists_dup_proc = pd.concat([playlists_dup_filter, playlists_no_dup], axis=0)
playlists_dup_proc = playlists_dup_proc.iloc[:, :6]
playlists_dup_proc.info()

# 2722+106515 =109237
# 114697 - 109237 = 타이틀 중복 제거로 5460행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109237 entries, 0 to 114696
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   tags          109237 non-null  object
 1   id            109237 non-null  int64 
 2   plylst_title  109237 non-null  object
 3   songs         109237 non-null  object
 4   like_cnt      109237 non-null  int64 
 5   updt_date     109237 non-null  object
dtypes: int64(2), object(4)
memory usage: 5.8+ MB


In [65]:
playlists_copy = playlists_dup_proc.copy()

In [66]:
playlists_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109237 entries, 0 to 114696
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   tags          109237 non-null  object
 1   id            109237 non-null  int64 
 2   plylst_title  109237 non-null  object
 3   songs         109237 non-null  object
 4   like_cnt      109237 non-null  int64 
 5   updt_date     109237 non-null  object
dtypes: int64(2), object(4)
memory usage: 5.8+ MB


> 3. 플레이리스트 별 노래 개수, 좋아요 개수 이상치 행 날리기

- 노래 개수 5~104개 가진 플레이리스트만 남기기 <- box & whisker plot(수염범위)

In [67]:
playlists_copy['songs_cnt'] = playlists_copy['songs'].apply(len)
playlists_copy['songs_cnt'].describe()

count    109237.000000
mean         45.949321
std          44.216494
min           1.000000
25%          19.000000
50%          29.000000
75%          53.000000
max         200.000000
Name: songs_cnt, dtype: float64

In [68]:
Q1 = 19
Q3 = 53
IQR = Q3 - Q1

print(Q1-(1.5*IQR))
print(Q3+(1.5*IQR))

-32.0
104.0


In [69]:
playlists_copy['songs_cnt'] = playlists_copy['songs'].apply(lambda x : len(x))
playlists_song_cut = playlists_copy[(playlists_copy['songs_cnt'] >= 5) & (playlists_copy['songs_cnt'] < 105) ]  # 5~104개만 남기기
playlists_song_cut.info() # 109237-98973 = 10264행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98973 entries, 0 to 114695
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          98973 non-null  object
 1   id            98973 non-null  int64 
 2   plylst_title  98973 non-null  object
 3   songs         98973 non-null  object
 4   like_cnt      98973 non-null  int64 
 5   updt_date     98973 non-null  object
 6   songs_cnt     98973 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 6.0+ MB


- like_cnt 7개 이하 삭제 (like_cnt 50% = 7)

In [70]:
playlists_song_cut['like_cnt'].describe()

count    98973.000000
mean        77.190284
std        437.540113
min          0.000000
25%          2.000000
50%          7.000000
75%         22.000000
max      53211.000000
Name: like_cnt, dtype: float64

In [71]:
Q1 = 2
Q3 = 22
IQR = Q3 - Q1

print(Q1-(1.5*IQR))
print(Q3+(1.5*IQR))

-28.0
52.0


In [72]:
playlists_song_like_cut = playlists_song_cut[playlists_song_cut['like_cnt'] >= 8]

In [73]:
playlists_song_like_cut = playlists_song_like_cut.iloc[:, :6]
playlists_song_like_cut.info()# 98973 - 48116 = 50857행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48116 entries, 8 to 114693
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          48116 non-null  object
 1   id            48116 non-null  int64 
 2   plylst_title  48116 non-null  object
 3   songs         48116 non-null  object
 4   like_cnt      48116 non-null  int64 
 5   updt_date     48116 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.6+ MB


In [74]:
playlists_copy_cnt = playlists_song_like_cut.copy()
playlists_copy_cnt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48116 entries, 8 to 114693
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          48116 non-null  object
 1   id            48116 non-null  int64 
 2   plylst_title  48116 non-null  object
 3   songs         48116 non-null  object
 4   like_cnt      48116 non-null  int64 
 5   updt_date     48116 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.6+ MB


> 4. song, 태그 빈도수에 따라 리스트에서 빼기 -> 리스트가 비워져있으면 행 날리기

- 노래 빈도수와 좋아요 상관관계

In [75]:
# 노래 별 좋아요 수 확인
ply_song_like = playlists_copy_cnt.explode('songs').groupby('songs').sum()['like_cnt']
ply_song_like = pd.DataFrame(ply_song_like).reset_index()
ply_song_like

C:\Users\cg063\AppData\Local\Temp\ipykernel_13660\3556108467.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ply_song_like = playlists_copy_cnt.explode('songs').groupby('songs').sum()['like_cnt']


,songs,like_cnt
0,0,27
1,3,1650
2,4,8
3,5,382
4,6,24
...,...,...
384351,707970,1309
384352,707978,642
384353,707979,72
384354,707980,8


In [76]:
# 노래 사용 빈도수 확인
ply_song_freq = playlists_copy_cnt.explode('songs').groupby('songs').count()['id']
ply_song_freq = pd.DataFrame(ply_song_freq).reset_index()
ply_song_freq.rename(columns={'id':'freq'}, inplace=True)
ply_song_freq

,songs,freq
0,0,1
1,3,5
2,4,1
3,5,2
4,6,1
...,...,...
384351,707970,3
384352,707978,2
384353,707979,2
384354,707980,1


In [77]:
playlists_copy_cnt[playlists_copy_cnt['songs'].apply(lambda x: 707978 in x)] # 2번 사용됨

,tags,id,plylst_title,songs,like_cnt,updt_date
17738,"[추울때, 어쿠스틱, 잔잔한, 새벽감성, 따뜻한, 티타임, 밤공기, 서정적인, 독서감상]",136386,독서할때 들으면 좋은 서정적이고 따듯한 팝송,"[562883, 696713, 537409, 464248, 430558, 59447...",19,2019-10-30 08:09:50.000
30493,"[감성, 겨울저녁, 새벽, 해외팝, 휴식, 팝]",11170,다가오는 겨울 새벽 내내 틀어놓고 싶은 노래,"[314302, 316407, 553161, 599409, 671675, 17680...",623,2019-11-25 09:55:34.000


In [78]:
ply_corr_song = pd.merge(ply_song_like, ply_song_freq, how='left', on='songs')
ply_corr_song

,songs,like_cnt,freq
0,0,27,1
1,3,1650,5
2,4,8,1
3,5,382,2
4,6,24,1
...,...,...,...
384351,707970,1309,3
384352,707978,642,2
384353,707979,72,2
384354,707980,8,1


In [79]:
ply_corr_song.describe()

,songs,like_cnt,freq
count,384356.000000,384356.000000,384356.000000
mean,353767.860039,965.093778,4.759996
std,204255.521387,3468.901774,14.915361
min,0.000000,8.000000,1.000000
25%,176775.500000,25.000000,1.000000
50%,353859.500000,112.000000,1.000000
75%,530712.250000,560.000000,3.000000
max,707986.000000,154102.000000,843.000000


In [80]:
ply_corr_song.corr()

,songs,like_cnt,freq
songs,1.000000,-0.001525,-0.000111
like_cnt,-0.001525,1.000000,0.787479
freq,-0.000111,0.787479,1.000000


In [116]:
import numpy as np

# ply_corr_tag DataFrame의 'column_name' 열의 80% 백분위수 확인
percentile_80 = np.percentile(ply_corr_song['freq'], 78)

print(percentile_80)

4.0


- 3회 이하 사용된 song, songs 리스트에서 빼기

In [81]:
# song usage count 계산
songs_counts = playlists_copy_cnt.explode('songs')['songs'].value_counts(ascending=True)

# 3번 이하 사용된 song
less_than_3_songs = list(songs_counts[songs_counts <= 3].index) # 296178

In [82]:
len(less_than_3_songs)

296178

In [83]:
pd.DataFrame(playlists_copy_cnt.explode('songs')['songs']).nunique() # 자르기 전 유니크 곡 : 384356

songs    384356
dtype: int64

In [84]:
# 3번 이하 사용된 songs 삭제 - hash function

from tqdm import tqdm
tqdm.pandas() 
playlists_copy_cnt['songs'] = playlists_copy_cnt['songs'].progress_apply(lambda songs: list(set(songs).difference(less_than_3_songs)))

  0%|          | 0/48116 [00:00<?, ?it/s]

100%|██████████| 48116/48116 [06:10<00:00, 129.87it/s]


In [85]:
# songs가 비어있지 않은 행만 남김
playlists_song_cut = playlists_copy_cnt[playlists_copy_cnt['songs'].apply(lambda x: len(x) > 0)]
playlists_song_cut.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
8,"[겨울, 크리스마스]",117162,12월의 재즈,"[677632, 309377, 473793, 179652, 407630, 43291...",353,2016-12-26 12:33:33.000
9,[발라드],151638,12월의 크리스마스를 yeah,"[560007, 246793, 591246, 653711, 112399, 40438...",29,2009-12-22 09:11:45.000
12,[일렉],27984,1월의 최신 EDM 소식,"[34851, 568611, 16261, 209982, 213895, 650671,...",84,2015-01-29 16:22:19.000
18,"[힙합, 랩]",48636,2000년대 힙합을 부탁해,"[168900, 514885, 395594, 82127, 449811, 181336...",54,2015-06-09 17:25:51.000
21,[락],30079,2008 그랜드민트 페스티벌,"[349057, 186245, 581896, 113676, 253458, 54453...",12,2008-12-23 19:56:46.000


In [86]:
playlists_song_cut['song_cnt'] = playlists_song_cut['songs'].apply(len)
playlists_song_cut['song_cnt'].unique()

C:\Users\cg063\AppData\Local\Temp\ipykernel_13660\558675112.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playlists_song_cut['song_cnt'] = playlists_song_cut['songs'].apply(len)


array([ 11,  33,   8,  20,  38,  58,  25,   6,  13,   1,  12,  14,  15,
        19,  28,  67,  17,  65,  21,  10,  70,  18,  32,  89,  62,  82,
        97,  72,  83,  37,  29,  34,  91,   4,   9,  26,  22,  90,  48,
        31,   7,  24,  16,  66,  57,  60,  45,  36,  51,  61,  64,  44,
        35,  87,   5,  47,  50,  39,  76,  80,  92,  63,  81,  54,   2,
        56,  75,  27,  30,  40,  96,  85,  23,  55,  93,  73,  43,  77,
        79,  86,  69,  95,  42,  71,  78,  59,   3,  49,  46,  53,  52,
        68,  94, 101,  88, 103,  84,  41,  98, 100,  74, 104,  99, 102],
      dtype=int64)

In [87]:
playlists_song_cut.explode('songs')['songs'].value_counts(ascending=True) # 3회 이하 사용된 song들이 알맞게 제거 되었는지 확인

159438      4
105987      4
201846      4
514106      4
301858      4
         ... 
418935    542
144663    573
366786    628
116573    689
205179    843
Name: songs, Length: 88178, dtype: int64

In [88]:
pd.DataFrame(playlists_song_cut.explode('songs')['songs']).nunique() # 384356개 -> 88178개

songs    88178
dtype: int64

In [89]:
playlists_song_cut = playlists_song_cut.iloc[:, :6]
playlists_song_cut.info()

# 48116 - 47191 = 925행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47191 entries, 8 to 114693
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          47191 non-null  object
 1   id            47191 non-null  int64 
 2   plylst_title  47191 non-null  object
 3   songs         47191 non-null  object
 4   like_cnt      47191 non-null  int64 
 5   updt_date     47191 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.5+ MB


- 3회 이하 사용된 tag, tags 리스트에서 빼기

In [90]:
pd.DataFrame(playlists_song_cut.explode('tags')['tags']).nunique() # 18094개

tags    18094
dtype: int64

In [91]:
# 태그 별 좋아요 수 확인
ply_tag_like = playlists_song_cut.explode('tags').groupby('tags').sum()['like_cnt']
ply_tag_like = pd.DataFrame(ply_tag_like).reset_index()
ply_tag_like

C:\Users\cg063\AppData\Local\Temp\ipykernel_13660\573314142.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ply_tag_like = playlists_song_cut.explode('tags').groupby('tags').sum()['like_cnt']


,tags,like_cnt
0,00,158
1,007,100
2,007시리즈,65
3,00s,10
4,00년,19
...,...,...
18089,힙해,2365
18090,힙힙힙,13
18091,힛뎀포크,41
18092,힛뎀폭,41


In [92]:
# 노래 사용 빈도수 확인
ply_tag_freq = playlists_song_cut.explode('tags').groupby('tags').count()['id']
ply_tag_freq = pd.DataFrame(ply_tag_freq).reset_index()
ply_tag_freq.rename(columns={'id':'freq'}, inplace=True)
ply_tag_freq

,tags,freq
0,00,3
1,007,2
2,007시리즈,2
3,00s,1
4,00년,1
...,...,...
18089,힙해,4
18090,힙힙힙,1
18091,힛뎀포크,1
18092,힛뎀폭,1


In [93]:
playlists_song_cut[playlists_song_cut['tags'].apply(lambda x: '007' in x)] # 2번 사용됨

,tags,id,plylst_title,songs,like_cnt,updt_date
692,"[액션히어로, 스파이영화, 본시리즈, OST, 007, 미션임파서블, 긴장감, 장르...",46937,긴장감 넘치는 액션과 사운드 첩보 스파이물 대표 OST,"[634628, 657608, 359929, 134688, 210978, 29181...",75,2017-07-13 10:23:13.000
75148,"[아델, 샘스미스, 제임스본드, 빌리아일리쉬, 007, 주제곡, 영화음악]",45506,007 제임스 본드 테마 음악 스페셜,"[594502, 568983, 515742, 333856, 291812, 18142...",25,2020-02-18 12:17:09.000


In [94]:
ply_corr_tag = pd.merge(ply_tag_like, ply_tag_freq, how='left', on='tags')
ply_corr_tag

,tags,like_cnt,freq
0,00,158,3
1,007,100,2
2,007시리즈,65,2
3,00s,10,1
4,00년,19,1
...,...,...,...
18089,힙해,2365,4
18090,힙힙힙,13,1
18091,힛뎀포크,41,1
18092,힛뎀폭,41,1


In [95]:
ply_corr_tag.describe()

,like_cnt,freq
count,1.809400e+04,18094.000000
mean,2.199057e+03,12.035813
std,2.641062e+04,128.797425
min,8.000000e+00,1.000000
25%,1.800000e+01,1.000000
50%,5.500000e+01,1.000000
75%,2.830000e+02,3.000000
max,1.501059e+06,6704.000000


In [96]:
ply_corr_tag.corr()

C:\Users\cg063\AppData\Local\Temp\ipykernel_13660\3957567245.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  ply_corr_tag.corr()


,like_cnt,freq
like_cnt,1.000000,0.971992
freq,0.971992,1.000000


In [109]:
import numpy as np

# ply_corr_tag DataFrame의 'column_name' 열의 80% 백분위수 확인
percentile_80 = np.percentile(ply_corr_tag['freq'], 80)

print(percentile_80)

3.0


In [97]:
# tag usage count 계산
tag_counts = playlists_song_cut.explode('tags')['tags'].value_counts(ascending=True)

# 3번 이하 사용된 태그
less_than_3_tags = list(tag_counts[tag_counts <= 3].index) # 14543

In [98]:
len(less_than_3_tags)

14543

In [99]:
from tqdm import tqdm
tqdm.pandas()

playlists_song_cut['tags'] = playlists_song_cut['tags'].progress_apply(lambda tags: list(set(tags).difference(less_than_3_tags)))

100%|██████████| 47191/47191 [00:45<00:00, 1044.83it/s]


In [100]:
# tags가 비어있지 않은 행만 남김
playlists_song_cut = playlists_song_cut[playlists_song_cut['tags'].apply(lambda x: len(x) > 0)]
playlists_song_cut.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
8,"[겨울, 크리스마스]",117162,12월의 재즈,"[677632, 309377, 473793, 179652, 407630, 43291...",353,2016-12-26 12:33:33.000
9,[발라드],151638,12월의 크리스마스를 yeah,"[560007, 246793, 591246, 653711, 112399, 40438...",29,2009-12-22 09:11:45.000
12,[일렉],27984,1월의 최신 EDM 소식,"[34851, 568611, 16261, 209982, 213895, 650671,...",84,2015-01-29 16:22:19.000
18,"[힙합, 랩]",48636,2000년대 힙합을 부탁해,"[168900, 514885, 395594, 82127, 449811, 181336...",54,2015-06-09 17:25:51.000
21,[락],30079,2008 그랜드민트 페스티벌,"[349057, 186245, 581896, 113676, 253458, 54453...",12,2008-12-23 19:56:46.000


In [101]:
playlists_song_cut['tag_cnt'] = playlists_song_cut['tags'].apply(len)
playlists_song_cut['tag_cnt'].unique()

array([ 2,  1,  4,  5,  3,  6,  9,  8, 10,  7, 11], dtype=int64)

In [102]:
playlists_song_cut.explode('tags')['tags'].value_counts(ascending=True)

DJJASON       2
집중력향상         4
Relax         4
신청곡           4
nct127        4
           ... 
잔잔한        3916
휴식         4286
감성         4438
드라이브       4464
기분전환       6703
Name: tags, Length: 3551, dtype: int64

In [103]:
pd.DataFrame(playlists_song_cut.explode('tags')['tags']).nunique() # 18094개 -> 3551개

tags    3551
dtype: int64

In [104]:
playlists_song_tag_cut = playlists_song_cut.iloc[:, :6]
playlists_song_tag_cut.info()

# 47191 - 46758  = 433행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46758 entries, 8 to 114693
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          46758 non-null  object
 1   id            46758 non-null  int64 
 2   plylst_title  46758 non-null  object
 3   songs         46758 non-null  object
 4   like_cnt      46758 non-null  int64 
 5   updt_date     46758 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.5+ MB


In [105]:
print(pd.DataFrame(playlists_song_tag_cut.explode('tags')['tags']).nunique()) # 18094개 -> 3551개
print(pd.DataFrame(playlists_song_tag_cut.explode('songs')['songs']).nunique()) # 384356개 -> 88178개

tags    3551
dtype: int64
songs    88178
dtype: int64


> 정리

=> song_meta : 날짜 수정, 장르 정보 없는 노래 제거  
=> playlists  
    - 타이틀 & 태그 불용어 삭제 : 특수문자, 빈 타이틀, 모두 숫자인 경우, 1글자인 경우 포함  
    - 타이틀 중복 제거 : 타이틀이 완벽하게 같을 경우 '1_노래 갯수 많은 순서 -> 2_좋아요 갯수 많은 순서 -> 3_태그 갯수 많은 순서 -> 4_최신 업데이트' 기준으로 1개씩 골라냄 (4가지를 했는데도 같은건 완벽히 같았기에 하나만 남김)  
    - 중복 제거 후 song_cnt describe 해 5~104개 남기기, like_cnt describe 8개 이상인 플레이리스트 남김 (50%=7)  
    - 3번 이하 사용된 tag, 3번 이하 사용된song 제거(75%=3) => 4번부터 남김
  
  
=> 115071(원래 playlist) -46758(전처리 후) = 68313행 삭제

> 데이터 저장

- 최종 전처리 파일은 ../0_data/tag3,song3_playlists.json 으로 저장

In [ ]:
# playlists 저장
playlists_song_tag_cut.to_json('../0_data/tag3,song3_playlists.json', orient='records')

---